In [10]:
import init
import epsilon_rho
import metrics
import optimize_algorithm as optimize
import numpy as np

In [11]:
def quantum_process_tomography_dephasing (num_qubits, gamma, num_loop, rho, kraus_operators):
    rho2 = epsilon_rho.calculate_dephasing(rho, num_qubits, gamma)
    # Optimizing
    out_kraus_operators = optimize.optimize_adam(rho, rho2, kraus_operators, num_qubits, 0.005, num_loop)

    _cost = 1 - metrics.compilation_trace_fidelity(rho, epsilon_rho.calculate_from_kraus_operators(rho2, out_kraus_operators))
    return rho2, out_kraus_operators, _cost


In [12]:
num_qubits = 3
rho = init.create_random_state(num_qubits=num_qubits)
unitary_matrix = init.create_unitary_matrix(num_qubits=num_qubits)
kraus_operators = init.create_kraus_operators_from_unitary(unitary_matrix=unitary_matrix) 

In [13]:
print (rho)

DensityMatrix([[0.0140127 +0.j, 0.01574752+0.j, 0.04741399+0.j,
                0.05895123+0.j, 0.03398523+0.j, 0.05554852+0.j,
                0.05881762+0.j, 0.01203673+0.j],
               [0.01574752+0.j, 0.01769711+0.j, 0.05328399+0.j,
                0.06624957+0.j, 0.03819271+0.j, 0.06242559+0.j,
                0.06609942+0.j, 0.01352691+0.j],
               [0.04741399+0.j, 0.05328399+0.j, 0.16043205+0.j,
                0.19946995+0.j, 0.11499391+0.j, 0.18795637+0.j,
                0.19901785+0.j, 0.04072799+0.j],
               [0.05895123+0.j, 0.06624957+0.j, 0.19946995+0.j,
                0.24800694+0.j, 0.14297537+0.j, 0.23369177+0.j,
                0.24744484+0.j, 0.05063832+0.j],
               [0.03398523+0.j, 0.03819271+0.j, 0.11499391+0.j,
                0.14297537+0.j, 0.08242493+0.j, 0.1347227 +0.j,
                0.14265131+0.j, 0.02919286+0.j],
               [0.05554852+0.j, 0.06242559+0.j, 0.18795637+0.j,
                0.23369177+0.j, 0.1347227 +0.j, 0.2

In [14]:
rho2, out_kraus_operators, _ = quantum_process_tomography_dephasing(num_qubits=num_qubits, 
                                                                    gamma=0.3, num_loop=700, 
                                                                    rho=rho, kraus_operators=kraus_operators)

In [15]:
rho2_kraus = epsilon_rho.calculate_from_kraus_operators(rho, out_kraus_operators)
#epsilon ^ dagger

In [16]:
metrics.compilation_trace_fidelity(rho2, np.transpose(np.conjugate(rho2_kraus)))

<tf.Tensor: shape=(), dtype=complex128, numpy=(0.9842247515415479+0j)>

In [17]:
#rho_2 vs rho_2 ^ dagger
#rho = rho
#rho' = rho2
#rho" = rho3

In [18]:
'''plotting
fix ite, run gamma & num-qubit
fix gamma, fix num-qubit, tính p' và p'2 dagger, 
lưu kraus và cost cuối cùng
'''
import os
np.set_printoptions(threshold=np.inf)
experiment_folder = ''
g_shift = 0.05

for num_qubits in range(5, 6):
    if (experiment_folder == ''):
        break
    g = 0
    rho = init.create_random_state(num_qubits=num_qubits)
    unitary_matrix = init.create_unitary_matrix(num_qubits=num_qubits)
    kraus_operators = init.create_kraus_operators_from_unitary(unitary_matrix=unitary_matrix)
    
    rho_file_path = os.path.join(experiment_folder, str(num_qubits)+"_qubits_rho.txt")
    with open(rho_file_path, 'w') as file:
        file.write(str(rho))
    
    while g < 1:
        folder_path = str(num_qubits)+"_qubits_"+"{:.3f}".format(g)
        folder_path = os.path.join(experiment_folder, folder_path)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        rho2, out_kraus_operators, _cost = quantum_process_tomography_dephasing(num_qubits, g, 700, rho, kraus_operators)
        
        rho2_file_path = os.path.join(folder_path, 'rho2.txt')
        kraus_file_path = os.path.join(folder_path, 'kraus_operators.txt')
        cost_file_path = os.path.join(folder_path, 'cost.txt')
        fidelity_file_path = os.path.join(folder_path, 'fidelity.txt')
        
        print(g, num_qubits, _cost)
        g = round(g + g_shift, 3)

        
        rho3 = epsilon_rho.calculate_from_kraus_operators(rho2, out_kraus_operators)
        with open(rho2_file_path, 'w') as file:
            file.write(str(rho2))
        with open(kraus_file_path, 'w') as file:
            file.write(str(out_kraus_operators.numpy()))
        with open(cost_file_path, 'w') as file:
            file.write(str(_cost.numpy().real))
        with open(fidelity_file_path, 'w') as file:
            file.write(str(1 - metrics.compilation_trace_fidelity(rho, rho3).numpy().real))

0 5 tf.Tensor((-2.326829351417814e-08+6.29296407548244e-25j), shape=(), dtype=complex128)
0.05 5 tf.Tensor((2.6857782664535534e-09+8.743682997510855e-25j), shape=(), dtype=complex128)
0.1 5 tf.Tensor((-2.378555130633231e-09+7.711977179074344e-25j), shape=(), dtype=complex128)
0.15 5 tf.Tensor((-8.898073833307762e-09+7.278030711569055e-25j), shape=(), dtype=complex128)
0.2 5 tf.Tensor((-8.134279916305331e-09+5.301851313595372e-25j), shape=(), dtype=complex128)
0.25 5 tf.Tensor((-5.8473312858353665e-09+6.636899788068782e-25j), shape=(), dtype=complex128)
0.3 5 tf.Tensor((8.467325063321596e-09+7.432182021058293e-25j), shape=(), dtype=complex128)
0.35 5 tf.Tensor((2.4289911526587105e-08+8.20110841120437e-25j), shape=(), dtype=complex128)
0.4 5 tf.Tensor((4.6020037225424915e-08+5.621537195436187e-25j), shape=(), dtype=complex128)
0.45 5 tf.Tensor((8.485299407556823e-08+6.47118772201353e-25j), shape=(), dtype=complex128)
0.5 5 tf.Tensor((1.6128356983280412e-07+7.698320024652706e-25j), shape=